# Import packages

In [1]:
import pandas as pd
import numpy as np
import os
from anndata import read_h5ad
from sklearn.neighbors import NearestNeighbors


import warnings
warnings.filterwarnings('ignore')
import sys

# sys.path.append('/data/yosef2/users/chenling/scVI/')
sys.path.append('../../utils/')
from annotations_utils import *
import scvi
print(scvi.__version__)

import os
os.getcwd()

import logging
import os
import pickle
from MulticoreTSNE import MulticoreTSNE as TSNE

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
from hyperopt import hp

from scvi.inference import UnsupervisedTrainer, AlternateSemiSupervisedTrainer, SemiSupervisedTrainer
from scvi.models import VAE, SCANVI
from copy import deepcopy

from scvi.dataset.anndataset import AnnDatasetFromAnnData

import scanpy as sc
sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)

from anndata import read_h5ad
from anndata import AnnData

import matplotlib

matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42
%matplotlib inline

n_epochs = 150
show_plot = True
test_mode = False
use_batches = True
use_cuda = True
lr = 1e-3
retrain=True


sc.logging.print_versions()

[2020-02-13 10:39:40,166] INFO - scvi._settings | Added StreamHandler with custom formatter to 'scvi' logger.


0.5.0
scanpy==1.4.5.post2 anndata==0.6.22.post1 umap==0.3.10 numpy==1.17.2 scipy==1.3.1 pandas==0.25.1 scikit-learn==0.21.3 statsmodels==0.10.1 python-igraph==0.7.1 louvain==0.6.1


# Load data

In [3]:
data_path = '../../data-objs/pilot1/FinalAnnotation/'

In [8]:
import glob
glob.glob(data_path+'*.imputed.h5ad')

['../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.endopancreas.mixed.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.blood.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.Pancreas.Stromal.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.bladder.mixed.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.Bladder.Endothelial.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.Pancreas.Endothelial.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.PancreasExocrine.mixed.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.Lung.mixed.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.epithelial.imputed.h5ad',
 '../../data-objs/pilot1/FinalAnnotation/CombineCompartments.scANVI.Lung.Immune.imputed.h5ad',
 '../../data-obj

In [39]:
adata = read_h5ad(data_path+'CombineCompartments.scANVI.all.imputed.h5ad')
adata

AnnData object with n_obs × n_vars = 68451 × 21301 
    obs: 'Pancreas Prediction', 'Compartment Prediction', 'Tissue Tech', '10X_plate', 'Manual Annotation Round 2', 'scANVI Prediction By Organ', 'Compartment', 'scANVI Prediction By Organ-Compartment', 'tissue', 'scANVI Prediction By Organ with Alternate Training', 'scANVI Prediction By Organ-Compartment with Alternate Training', 'batch_id', 'Smoothed Compartment Prediction', 'S_score', 'G2M_score', 'phase', 'scANVI Prediction Ontology Term By Organ', 'scANVI Prediction Ontology Term By Organ with Alternate Training', 'scANVI Prediction Ontology Term By Organ-Compartment', 'scANVI Prediction Ontology Term By Organ-Compartment with Alternate Training', 'Manual Annotation Round 2 Ontology Terms', 'n_genes', 'log1p_total_counts', 'leiden'
    var: 'n_cells-0', 'n_cells-1'
    uns: 'Compartment Prediction_colors', 'Pancreas Prediction_colors', 'Smoothed Compartment Prediction_colors', 'leiden', 'neighbors', 'smooth_comp2_colors', 'smooth_

In [40]:
adata.obs['tissue'] = adata.obs['tissue'].str.lower()
adata.obs['subtissue'] = adata.obs['tissue']
adata.obs.loc[adata.obs['tissue'].isin(
    ['exopancreas1','endopancreas','exopancreas2','pancreasendocrine', 'pancreasexocrine']),'tissue'] = 'pancreas'

adata.obs['method'] = [tt.split('_')[1] for tt in adata.obs['Tissue Tech']]

adata.obs = adata.obs.drop(
    ['Compartment Prediction', '10X_plate','Compartment', 'S_score', 'G2M_score', 'phase'],
    axis=1)

adata.obs = adata.obs.rename(columns={"Tissue Tech": "tissue_method",
                                      "Pancreas Prediction" : "pancreas_prediction",
                                     "Manual Annotation Round 2": "manual_free_annotation",
                                     "Manual Annotation Round 2 Ontology Terms": "manual_cell_ontology",
                                     "Smoothed Compartment Prediction": "compartment_prediction",
                                     "scANVI Prediction By Organ": "scANVI_organ_free_annotation",
                                     "scANVI Prediction By Organ-Compartment": "scANVI_organ_compartment_free_annotation",
                                     "scANVI Prediction By Organ with Alternate Training": "scANVI_organ_alternate_training_free_annotation",
                                     "scANVI Prediction By Organ-Compartment with Alternate Training": "scANVI_organ_compartment_alternate_training_free_annotation",
                                     "scANVI Prediction Ontology Term By Organ": "scANVI_organ_cell_ontology",
                                     "scANVI Prediction Ontology Term By Organ-Compartment": "scANVI_organ_compartment_cell_ontology",
                                     "scANVI Prediction Ontology Term By Organ with Alternate Training": "scANVI_organ_alternate_training_cell_ontology",
                                     "scANVI Prediction Ontology Term By Organ-Compartment with Alternate Training": "scANVI_organ_compartment_alternate_training_cell_ontology"})

adata.obs['compartment_prediction'] = adata.obs['compartment_prediction'].str.lower()

adata.obs = adata.obs.reindex(sorted(adata.obs.columns), axis=1)

adata

AnnData object with n_obs × n_vars = 68451 × 21301 
    obs: 'batch_id', 'compartment_prediction', 'leiden', 'log1p_total_counts', 'manual_cell_ontology', 'manual_free_annotation', 'method', 'n_genes', 'pancreas_prediction', 'scANVI_organ_alternate_training_cell_ontology', 'scANVI_organ_alternate_training_free_annotation', 'scANVI_organ_cell_ontology', 'scANVI_organ_compartment_alternate_training_cell_ontology', 'scANVI_organ_compartment_alternate_training_free_annotation', 'scANVI_organ_compartment_cell_ontology', 'scANVI_organ_compartment_free_annotation', 'scANVI_organ_free_annotation', 'subtissue', 'tissue', 'tissue_method'
    var: 'n_cells-0', 'n_cells-1'
    uns: 'Compartment Prediction_colors', 'Pancreas Prediction_colors', 'Smoothed Compartment Prediction_colors', 'leiden', 'neighbors', 'smooth_comp2_colors', 'smooth_comp_colors'
    obsm: 'X_scvi', 'X_tsne', 'X_umap'

In [43]:
set(adata.obs['pancreas_prediction']),set(adata.obs['compartment_prediction'])

({'not pancreas', 'pancreas'},
 {'endothelial', 'epithelial', 'immune', 'mixed', 'stromal'})

# Write objs for browser

## Write compartment objs

In [34]:
for c in list(set(adata.obs['compartment_prediction'])):
    adata_comp = adata[adata.obs['compartment_prediction']==c].copy()
    
    if adata_comp.shape[0]>0:
        sc.pp.pca(adata_comp)
        sc.pp.neighbors(adata_comp, n_neighbors=20, n_pcs=30, use_rep="X_scvi")
        sc.tl.umap(adata_comp)
        sc.tl.leiden(adata_comp)
        sc.tl.tsne(adata_comp, use_rep = 'X_scvi')
        adata_comp.write_h5ad('../../data-objs/pilot1/TSP1_objs_browser/TSP1_all_'+c+'.h5ad')
    else:
        print("there is no cells to put in "+c)

computing PCA with n_comps = 50
    finished (0:00:22)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:07)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:32)


... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


computing PCA with n_comps = 50
    finished (0:00:04)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:03)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:12)


... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


computing PCA with n_comps = 50
    finished (0:00:02)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:02)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:18)


... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


computing PCA with n_comps = 50
    finished (0:00:03)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:01)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:16)


... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


computing PCA with n_comps = 50
    finished (0:00:01)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:00)
computing UMAP
    finished: added
    'X_umap', UMAP coordinates (adata.obsm) (0:00:11)


... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


## Write tissue objs

In [44]:
for t in list(set(adata.obs['tissue'])):
    adata_tiss = adata[adata.obs['tissue']==t].copy()
    
    if adata_tiss.shape[0]>0:
        sc.pp.pca(adata_tiss)
        sc.pp.neighbors(adata_tiss, n_neighbors=20, n_pcs=30, use_rep="X_scvi")
#         sc.tl.umap(adata_tiss)
#         sc.tl.leiden(adata_tiss)
#         sc.tl.tsne(adata_tiss, use_rep = 'X_scvi')
        adata_tiss.write_h5ad('../../data-objs/pilot1/TSP1_objs_browser/TSP1_'+t+'.h5ad')
    else:
        print("there is no cells to put in "+t)

computing PCA with n_comps = 50
    finished (0:00:07)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:04)


... storing 'compartment_prediction' as categorical
... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


computing PCA with n_comps = 50
    finished (0:00:02)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:01)


... storing 'compartment_prediction' as categorical
... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


computing PCA with n_comps = 50
    finished (0:00:05)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:03)


... storing 'compartment_prediction' as categorical
... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


computing PCA with n_comps = 50
    finished (0:00:03)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:02)


... storing 'compartment_prediction' as categorical
... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


computing PCA with n_comps = 50
    finished (0:00:06)
computing neighbors
    finished: added to `.uns['neighbors']`
    'distances', distances for each pair of neighbors
    'connectivities', weighted adjacency matrix (0:00:04)


... storing 'compartment_prediction' as categorical
... storing 'method' as categorical
... storing 'subtissue' as categorical
... storing 'tissue' as categorical


## Write tissue_compartment objs

In [ ]:
for t in list(set(adata.obs['tissue'])):
    adata_tiss = adata[adata.obs['tissue']==t].copy()
    if adata_tiss.shape[0]>0:
        
        for c in list(set(adata_tissue.obs['compartment_prediction'])):
            adata_comp = adata_tissue[adata_tissue.obs['compartment_prediction']==c].copy()

            if adata_comp.shape[0]>0:
                sc.pp.pca(adata_comp)
                sc.pp.neighbors(adata_comp, n_neighbors=20, n_pcs=30, use_rep="X_scvi")
                sc.tl.umap(adata_comp)
                sc.tl.leiden(adata_comp)
                sc.tl.tsne(adata_comp, use_rep = 'X_scvi')
                adata_comp.write_h5ad('../../data-objs/pilot1/TSP1_objs_browser/TSP1_'+t+'_'+c'.h5ad')
            else:
                print("there is no cells to put in "+t+" "+c)
    
    
    else:
        print("there is no cells to put in "+t)

# write files

In [48]:
raw.write_h5ad('CombineCompartments.scANVI.all.count.h5ad')
imputed_adata.write_h5ad('CombineCompartments.scANVI.all.imputed.h5ad')
new_adata.write_h5ad('CombineCompartments.scANVI.all.scanpy_norm.h5ad')

... storing 'Manual Annotation Round 2' as categorical
... storing 'scANVI Prediction By Organ' as categorical
... storing 'scANVI Prediction By Organ-Compartment' as categorical
... storing 'tissue' as categorical
... storing 'scANVI Prediction By Organ with Alternate Training' as categorical
... storing 'scANVI Prediction By Organ-Compartment with Alternate Training' as categorical
... storing 'phase' as categorical
... storing 'scANVI Prediction Ontology Term By Organ' as categorical
... storing 'scANVI Prediction Ontology Term By Organ with Alternate Training' as categorical
... storing 'scANVI Prediction Ontology Term By Organ-Compartment' as categorical
... storing 'scANVI Prediction Ontology Term By Organ-Compartment with Alternate Training' as categorical
... storing 'Manual Annotation Round 2 Ontology Terms' as categorical


In [49]:
def SubsetWrite(adata, subset, filename):
    adata = adata[subset]
    if adata.shape[0]>0:
        print(filename)
        sc.pp.neighbors(adata, n_neighbors=20, n_pcs=30, use_rep="X_scvi")
        sc.tl.umap(adata)
        sc.tl.leiden(adata)
        sc.tl.tsne(adata, use_rep = 'X_scvi')
        adata.write_h5ad(filename)
    else:
        print("there is no cells to put in %s" %filename)
        
        
normalization_method = ['count', 'imputed','scanpy_norm']


In [ ]:
for comp in np.unique(raw.obs['Smoothed Compartment Prediction']):
    subset = raw.obs['Smoothed Compartment Prediction']==comp
    for norm, adata in zip(normalization_method, [raw, imputed_adata, new_adata]):
        SubsetWrite(adata, subset, 'CombineCompartments.scANVI.%s.%s.h5ad'%(comp, norm))
    

CombineCompartments.scANVI.Immune.count.h5ad
CombineCompartments.scANVI.Immune.imputed.h5ad
CombineCompartments.scANVI.Immune.scanpy_norm.h5ad
CombineCompartments.scANVI.endothelial.count.h5ad
CombineCompartments.scANVI.endothelial.imputed.h5ad
CombineCompartments.scANVI.endothelial.scanpy_norm.h5ad
CombineCompartments.scANVI.epithelial.count.h5ad
CombineCompartments.scANVI.epithelial.imputed.h5ad
CombineCompartments.scANVI.epithelial.scanpy_norm.h5ad
CombineCompartments.scANVI.mixed.count.h5ad
CombineCompartments.scANVI.mixed.imputed.h5ad
CombineCompartments.scANVI.mixed.scanpy_norm.h5ad


In [ ]:
for tissue in np.unique(raw.obs['tissue']):
    subset = raw.obs['tissue']==tissue
    for norm, adata in zip(normalization_method, [raw, imputed_adata, new_adata]):
        SubsetWrite(adata, subset, 'CombineCompartments.scANVI.%s.%s.h5ad'%(tissue, norm))
    

In [ ]:
for tissue in np.unique(raw.obs['tissue']):
    for comp in ['Endothelial', 'Epithelial', 'Immune', 'Stromal', 'mixed']:
        subset = np.logical_and(raw.obs['Smoothed Compartment Prediction']==comp, raw.obs['tissue']==tissue)
        for norm, adata in zip(normalization_method, [raw, imputed_adata, new_adata]):
            SubsetWrite(adata, subset, 'CombineCompartments.scANVI.%s.%s.%s.h5ad'%(tissue, comp, norm))

    

In [ ]:
subset = np.logical_or(raw.obs['Pancreas Prediction']=='pancreas', raw.obs['tissue']=='Pancreas')
for norm, adata in zip(normalization_method, [raw, imputed_adata, new_adata]):
    SubsetWrite(adata, subset, 'CombineCompartments.scANVI.%s.%s.h5ad'%('pancreas_all', norm))
